In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/Colab\ Notebooks/GCN

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/GCN


In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 4.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626497 sha256=d4ed76c4dff01c671ec7d303c97597b061dc58362f6e178df6e3df8696de18a4
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset, KNNBasic, NormalPredictor,BaselineOnly,KNNWithMeans,KNNBaseline
from surprise import SVD, SVDpp, NMF, SlopeOne, CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import accuracy
import random
import time

In [4]:
# read movielens .dat file
# df_raw = [i.strip().split('::') for i in open('ml-1m-2/ratings.dat').readlines()] # 1M data
df_raw = [i.strip().split() for i in open('ml-100k/u.data').readlines()] # 100k data
df = pd.DataFrame(df_raw, columns=['userID', 'itemID', 'rating', 'timestamp'])
df.drop('timestamp', axis=1, inplace=True)
df['rating'] = df['rating'].astype(int)

In [5]:
np.min(df.rating), np.max(df.rating)

(1, 5)

In [6]:
train, test = train_test_split(df, test_size=0.15, random_state=14, stratify=df['userID']) # initial, seeded split

reader = Reader(rating_scale=(1,5))
train_data = Dataset.load_from_df(train, reader).build_full_trainset()

# svd_algo = SVD(n_epochs=100, lr_all=0.0005, reg_all=0.01) # for 1M data
svd_algo = SVD(n_epochs=100, lr_all=0.0025, reg_all=0.1) # for 100k data
start = time.time()
svd_algo.fit(train_data)
stop = time.time()
print(f'Training took {stop-start}s')

Training took 7.646852254867554s


In [7]:
# Compute unbiased accuracy on test
testset = list(test.itertuples(index=False, name=None))
predictions = svd_algo.test(testset)
print(f'Unbiased accuracy on test set = {accuracy.rmse(predictions)}')

RMSE: 0.9117
Unbiased accuracy on test set = 0.9116539455359164


In [8]:
def return_unrated(train_data, userID, user_col='userID', item_col='itemID'):
    unique_items = train_data[item_col].unique()
    user_rated_items = train_data.loc[train_data[user_col]==userID, item_col].unique()

    return unique_items[~np.in1d(unique_items, user_rated_items)]

def hit_ratio(df, test, model, k=5):
    unique_test_users = test['userID'].unique()
    hr_k = 0
    user_count = 0
    for uid in unique_test_users:
        user_not_rated = return_unrated(df, uid).tolist()
        user_rel_items = test.loc[test['userID']==uid, 'itemID'].values.tolist()
        item_predictions = []
        for iid in user_not_rated:
            item_predictions.append([iid, model.predict(uid, iid).est])

        item_predictions = pd.DataFrame(item_predictions, columns=['itemID', 'predicted_rating']).sort_values(by='predicted_rating', ascending=False)
        sum_hits = sum(item_predictions[:k]['itemID'].isin(user_rel_items))
        if sum_hits > 0:
            hr_k += 1
        user_count +=1

    return hr_k/user_count

hr_20 = hit_ratio(train, test, svd_algo, k=20)

In [9]:
hr_20

0.49522799575821846

In [ ]:
# our_seed = 14

# #Define a Reader object
# reader = Reader(rating_scale=(1,5)) # use rating min and max here for scale

# #Create the dataset to be used for building the filter
# data = Dataset.load_from_df(df, reader)

# #get the raw ratings
# raw_ratings = data.raw_ratings

# # shuffle ratings
# random.seed(our_seed)
# np.random.seed(our_seed)
# random.shuffle(raw_ratings)

# #A = 85% of the data, B = 15% of the data
# threshold = int(.85 * len(raw_ratings))
# A_raw_ratings = raw_ratings[:threshold]
# B_raw_ratings = raw_ratings[threshold:]

# data.raw_ratings = A_raw_ratings  # data is now the set A

# # Grid search
# param_grid = {'n_epochs': [100],
# 			  'lr_all': [.001, .0005],
# 			  'reg_all': [.1, .01, .001]}
# grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], refit=True, cv=5)
# grid_search.fit(data)

# # svd_algo = grid_search.best_estimator['rmse']
# svd_algo = SVD(n_epochs=100)

# # retrain on the whole set A
# trainset = data.build_full_trainset()
# svd_algo.fit(trainset)

In [ ]:
# Compute biased accuracy on A 
# predictions = svd_algo.test(trainset.build_testset())
# print(f'Biased accuracy on A = {accuracy.rmse(predictions)}')

RMSE: 0.4245
Biased accuracy on A = 0.42451647573858564


In [ ]:
# # Compute unbiased accuracy on B
# testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
# predictions = svd_algo.test(testset)
# print(f'Unbiased accuracy on B = {accuracy.rmse(predictions)}')

RMSE: 0.9191
Unbiased accuracy on B = 0.9190505596823432


In [ ]:
# grid_search.best_params

{'rmse': {'n_epochs': 100, 'lr_all': 0.0005, 'reg_all': 0.01}}